## 问答意图识别（进阶方向）


### 任务
- 任务说明：使用文本相似度和prompt进行意图识别
- 任务要求：
    - 计算提问与现有文档的相似度
    - 构造prompt完成意图识别
- 打卡要求：完成RAG完整流程，并提交结果进行打分

### 相关代码

In [1]:
import time 
import jwt
import requests
import jieba
import re
from tqdm import tqdm
import json
import pdfplumber
from langchain.schema import Document
from rank_bm25 import BM25Okapi
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
 

def extract_page_text(filepath, max_len=256, overlap_len=100):
    page_content  = []
    pdf =pdfplumber.open(filepath)
    page_count = 0
    # pattern = r'^\d{1,3}'
    for page in tqdm(pdf.pages):
        page_text = page.extract_text().strip()
        raw_text = [text.strip() for text in page_text.split('\n')]
        new_text = '\n'.join(raw_text)
        new_text = re.sub(r'\n\d{2,3}\s?', '\n', new_text)
        # new_text = re.sub(pattern, '', new_text).strip()
        if len(new_text)>10 and '..............' not in new_text:
            page_content.append(new_text)
        else:
            page_content.append('  ')

    cleaned_chunks = []
    i = 0
    all_str = ''.join(page_content)
    all_str = all_str.replace('\n', '')
    while i<len(all_str):
        cur_s = all_str[i:i+max_len]
        if len(cur_s)>10:
            cleaned_chunks.append(Document(page_content=cur_s, metadata={'page':page_count+1}))
        i+=(max_len - overlap_len)

    return cleaned_chunks,page_content
# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token("f1a0b6c3d36d46d3eed74a6c7de3e9e4.pZ88EkbBscyHXXcJ", 1000)
    }

    data = {
        "model": "glm-4",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()



def get_answer_from_llm(question_idx,questions):
    for query_idx in question_idx:
        doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
        max_score_page_idxs = doc_scores.argsort()[-3:]

        pairs = []
        for idx in max_score_page_idxs:
            pairs.append([questions[query_idx]["question"], pdf_content[idx] ])

        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            inputs = {key: inputs[key].cuda() for key in inputs.keys()}
            scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
        max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
        questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

        prompt= '''你是一个汽车专家，帮我结合给定的资料，回答所给的问题。如果问题无法从资料中获得，请输出:结合给定的资料，无法回答问题。
    资料：{0}

    问题：{1}
        '''.format(
            ''.join([f'第{i+1}页内容：' + pdf_content[i].replace('\n', '') + '\n' for i in doc_scores.argsort()[-3:]]) ,
            questions[query_idx]["question"]
        )
        prompt2= '''你是一个汽车专家，帮我结合你的经验回答所给的问题.请把回复内容控制在100字以内!
    问题：{0}
        '''.format(
            
            questions[query_idx]["question"]
        )
        answer = ask_glm(prompt)['choices'][0]['message']['content']
        # if '无法回答' in answer:
        #     answer = ask_glm(prompt2)['choices'][0]['message']['content']
        questions[query_idx]['answer'] = answer
        print(query_idx,questions[query_idx])
    return questions

def get_questions_from_file(file_path):
    with open(f'/root/code/submit_task8_glm4.json', 'r', encoding='utf8') as f:
        questions = f.read()
    questions = eval(questions)
    return questions

def get_useless_question_idx(questions):
    ## 收集无法回答答案的问题的索引
    useless_question_idx = []
    for i,question in enumerate(questions):
        if '无法回答' in question['answer']:
            # print(question)
            useless_question_idx.append(i)
    return useless_question_idx

file_path = '/root/code/submit_task8_glm4.json'




questions = json.load(open("./data/questions.json"))
filepath = './data/初赛训练数据集.pdf'
_,pdf_content = extract_page_text(filepath, max_len=256, overlap_len=100)


tokenizer = AutoTokenizer.from_pretrained('/root/code/quietnight/bge-reranker-large/')
rerank_model = AutoModelForSequenceClassification.from_pretrained('/root/code/quietnight/bge-reranker-large/')
rerank_model.cuda()


pdf_content_words = [jieba.lcut(x ) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

questions = get_answer_from_llm(range(5),questions)


with open(f'submit_task8_glm4.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: './data/questions.json'

### 意图识别

In [3]:
prompt = """
你是一个汽车方面的专家，请判断下面的提问回答是否与汽车使用相关。只能回答相关或者是不相关，不要回答其他内容
问题是:
{}
"""
for question in questions[:5]:
    answer = ask_glm(prompt.format(question['question']))['choices'][0]['message']['content']
    question['意图'] = answer
print(questions[:5])

[{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '前排座椅通风的相关内容在第115页和第117页。', 'reference': 'page_117', '意图': '相关'}, {'question': '"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"', 'answer': '在第123页和第124页可以找到关于车辆的儿童安全座椅固定装置的相关内容。', 'reference': 'page_123', '意图': '相关'}, {'question': '“打开前机舱盖”的相关信息在第几页？', 'answer': '“打开前机舱盖”的相关信息在第308页和第307页。在第308页中，有详细的步骤说明如何抬起和关闭前机舱盖，并包含了相关的安全警告。在第307页中，提到了在打开前机舱盖前需要确保无障碍物，并说明了如何操作前机舱盖的开启拉手。', 'reference': 'page_307', '意图': '相关'}, {'question': '“打开前机舱盖”这个操作在哪一页？', 'answer': '打开前机舱盖的操作描述在第308页和第307页中给出。这两页内容都涉及到了如何打开前机舱盖的步骤和注意事项。', 'reference': 'page_307', '意图': '相关'}, {'question': '“查看行车记录仪视频”这一项内容在第几页？', 'answer': '第275页和第276页。在这两页的内容中，都有提到如何通过中央显示屏查看行车记录仪视频的步骤和说明。', 'reference': 'page_275', '意图': '相关。这个问题涉及到行车记录仪的使用，是汽车使用相关的查询。'}]


### 页码问题识别

In [4]:
yema_prompt= """
你是一个汽车方面的专家，请判断下面的提问,回答是否涉及到文档的页码信息,只能回答涉及或者是不涉及，不要回答其他内容
问题是:
{}
"""
for question in questions[:5]:
    if '页' in question['question']:
        answer = '涉及'
    else:
        answer = '不涉及'
    # answer = ask_glm(yema_prompt.format(question['question']))['choices'][0]['message']['content']
    question['页码'] = answer
    print(question)


{'question': '“前排座椅通风”的相关内容在第几页？', 'answer': '前排座椅通风的相关内容在第115页和第117页。', 'reference': 'page_117', '意图': '相关', '页码': '涉及'}
{'question': '"关于车辆的儿童安全座椅固定装置，在哪一页可以找到相关内容？"', 'answer': '在第123页和第124页可以找到关于车辆的儿童安全座椅固定装置的相关内容。', 'reference': 'page_123', '意图': '相关', '页码': '涉及'}
{'question': '“打开前机舱盖”的相关信息在第几页？', 'answer': '“打开前机舱盖”的相关信息在第308页和第307页。在第308页中，有详细的步骤说明如何抬起和关闭前机舱盖，并包含了相关的安全警告。在第307页中，提到了在打开前机舱盖前需要确保无障碍物，并说明了如何操作前机舱盖的开启拉手。', 'reference': 'page_307', '意图': '相关', '页码': '涉及'}
{'question': '“打开前机舱盖”这个操作在哪一页？', 'answer': '打开前机舱盖的操作描述在第308页和第307页中给出。这两页内容都涉及到了如何打开前机舱盖的步骤和注意事项。', 'reference': 'page_307', '意图': '相关', '页码': '涉及'}
{'question': '“查看行车记录仪视频”这一项内容在第几页？', 'answer': '第275页和第276页。在这两页的内容中，都有提到如何通过中央显示屏查看行车记录仪视频的步骤和说明。', 'reference': 'page_275', '意图': '相关。这个问题涉及到行车记录仪的使用，是汽车使用相关的查询。', '页码': '涉及'}


### 总结
优化点：
- 使用了重排序，将与问题相关性最高的TOP3的内容 同时输入给模型，提升了数据的丰富性
- 对于每一个页面的内容，添加了页码信息，因此模型回答的时候能够找到对应的页码

结果：
- GLM3 得分0.73
- GBM4 得分0.75
- GBM4 对模型无法回答出来的 根据本身经验回答 得分 0.72
- GBM4 对相关性 根据本身经验回答 得分 0.71




### 参考资料

[向量数据库Chroma极简教程](https://mp.weixin.qq.com/s/SCCEAZqmKypSE4K-JE4euA)